In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
import random
import logging
import csv
import jieba
from chatterbot import ChatBot
from chatterbot.trainers import ChatterBotCorpusTrainer

logging.basicConfig(level=logging.CRITICAL)

#  jieba.load_userdict("../jieba_dict/av.txt")
#  jieba.load_userdict("../jieba_dict/dict.txt")
#  jieba.load_userdict("../jieba_dict/ptt_words.txt")
#  jieba.load_userdict("../jieba_dict/taiwan_actor.txt")
#  jieba.load_userdict("../jieba_dict/taiwan_name.txt")
#  jieba.load_userdict("../jieba_dict/usa_name.txt")
stopwords = []
can_words = []

with open("data/stopwords/stopwords", 'r') as f:
    stopwords = [i.replace(" ", "") for i in f.read().split("\n")]
with open("data/cans/cans.txt", "r") as f:
    can_words = f.read().split("\n")
bot = ChatBot(
    "I wonna sleep....",
    storage_adapter='chatterbot.storage.SQLStorageAdapter',
    logic_adapters=[
        "chatterbot.logic.MathematicalEvaluation",
        "chatterbot.logic.BestMatch",
        #  {
        #  'import_path': 'chatterbot.logic.LowConfidenceAdapter',
        #  'threshold': 0.5,
        #  'default_response': '我是不知道你在供三小啦 但我不想人生攻擊你'
        #  }
    ],
    filters=[
        'chatterbot.filters.RepetitiveResponseFilter'
    ],
    language="chinese",
)


#print("Type something to begin...")


def remove_stopwords(words):
    for i in stopwords:
        if i in words:
            words.remove(i)
    return words

def getAnswer(question):
#while True:
    #try:
        #__input = input()
        answer=''
        with open("data/keywords.csv", 'r') as f:
            reader = csv.DictReader(f)
            dont_use_chatter = False
            for row in reader:
                if row["QUESTION"] in question:
                    logging.debug("the words in my db")
                    print(row["ANSWER"])
                    answer=str(row["ANSWER"])
                    dont_use_chatter = True
                    break
            if dont_use_chatter == False:
                bot_input = bot.get_response(question)
                logging.debug("confidence" + str(bot_input.confidence))
                if bot_input.confidence > 0.5:
                    print(bot_input)
                    answer=str(bot_input)
                    dont_use_chatter = True
                else:
                    logging.debug("confidence too low, try to search")
                    segs = jieba.cut(question)
                    dont_use_chatter = True
                    segged = list(segs)
                    without_sw = remove_stopwords(segged)
                    logging.debug("without stopword: " + "".join(without_sw))
                    new_bot_input = random.choice(can_words).replace("%p", question)
                    print(new_bot_input)
                    answer=str(new_bot_input)
            #print answer
    #except (KeyboardInterrupt, EOFError, SystemExit):
        #break


In [5]:
getAnswer('這次颱風會放假嗎？')

現在行政院長換賴清德全台都不用放假啦


In [6]:
getAnswer('核廢料應該要放哪裡？')

放佛光山


In [7]:
getAnswer('這場戰爭回來，我就要結婚了？')

所謂結婚，就是你會持續錯上一輩子


In [8]:
getAnswer('你是擁核人？')

什麼永和，我早就搬去土城了


In [9]:
getAnswer('遠傳的訊號強嗎？')

只有遠傳沒有訊息
